In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from nbconvert import PythonExporter
import os
from tqdm import tqdm
from PIL import Image
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, log_loss
import time
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from tensorflow.keras.applications.xception import Xception
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import callbacks, regularizers
from tensorflow.io import read_file
from tensorflow.image import decode_jpeg, resize
from tensorflow import one_hot, cast, float32
from keras.applications.xception import preprocess_input

2025-09-09 08:52:03.032308: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757407923.212856      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757407923.269978      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


#### Loading data ####

In [4]:
# adding the images, labels, styles
file_path = "/kaggle/input/wikiart-all-artpieces/wikiart_art_pieces.csv"
df = pd.read_csv(file_path)
df.head()

,artist,style,genre,movement,tags,url,img,file_name
0,Byzantine Mosaics,Early Byzantine (c. 330–750),religious painting,Byzantine Art,"['Holyplaces', 'Byzantinearchitecture', 'Arch'...",https://www.wikiart.org/en/byzantine-mosaics/e...,https://uploads2.wikiart.org/00211/images/byza...,0-ravenna-cappella-arcivescovile-166.jpg
1,Byzantine Mosaics,Early Byzantine (c. 330–750),religious painting,Byzantine Art,"['Holyplaces', 'Byzantinearchitecture', 'Arch']",https://www.wikiart.org/en/byzantine-mosaics/e...,https://uploads2.wikiart.org/00211/images/byza...,1-ravenna-cappella-arcivescovile-167.jpg
2,Byzantine Mosaics,Early Byzantine (c. 330–750),religious painting,Byzantine Art,"['Prophet', 'History']",https://www.wikiart.org/en/byzantine-mosaics/e...,https://uploads2.wikiart.org/00211/images/byza...,2-ravenna-cappella-arcivescovile-168.jpg
3,Byzantine Mosaics,Early Byzantine (c. 330–750),religious painting,Byzantine Art,"['Holyplaces', 'Prophet']",https://www.wikiart.org/en/byzantine-mosaics/e...,https://uploads2.wikiart.org/00211/images/byza...,3-ravenna-cappella-arcivescovile-169.jpg
4,Byzantine Mosaics,Early Byzantine (c. 330–750),religious painting,Byzantine Art,"['Holyplaces', 'Prophet']",https://www.wikiart.org/en/byzantine-mosaics/e...,https://uploads2.wikiart.org/00211/images/byza...,4-ravenna-cappella-arcivescovile-171.jpg


In [5]:
df.shape

(176436, 8)

In [6]:
def filter_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """clean the dataframe to keep categories wanted"""

    #lists :
    styles_to_keep = ['Orientalism', 'Neoclassicism', 'Naturalism', 'Romanticism', 'Academicism', 'Neo-Rococo', 'Kitsch', 'Realism', 'Impressionism', 'Expressionism', 'Japonism', 'Magic Realism', 'Symbolism', 'Costumbrismo', 'Biedermeier', 'Luminism', 'Ink and wash painting', 'Naïve Art (Primitivism)', 'Art Nouveau (Modern)', 'Divisionism', 'Social Realism', 'Tonalism', 'Post-Impressionism', 'Pointillism', 'Ukiyo-e', 'American Realism', 'Socialist Realism', 'Fauvism', 'Cubism', 'Abstract Art', 'Precisionism', 'Figurative Expressionism', 'Existential Art', 'Surrealism', 'New Realism', 'Neo-baroque', 'Abstract Expressionism', 'Neo-Romanticism', 'Neo-Impressionism', 'Cloisonnism', 'Synthetism', 'Regionalism', 'Orphism', 'Art Deco', 'Neo-Expressionism', 'Fantasy Art', 'New Medievialism', 'Neo-Byzantine', 'Lyrical Abstraction', 'New Ink Painting', 'Verism', 'Contemporary Realism', 'Color Field Painting', 'Art Brut', 'Outsider art', 'Cubo-Expressionism', 'Pop Art', 'Geometric', 'Art Singulier', 'Action painting', 'Constructivism', 'Neoplasticism', 'Intimism', 'Dada', 'Analytical Cubism', 'Synthetic Cubism', 'Muralism', 'Futurism', 'Cubo-Futurism', 'Tachisme', 'Modernismo', 'Metaphysical art', 'Excessivism', 'Classical Realism', 'Severe Style', 'Miserablism', 'Art Informel', 'Neo-Pop Art', 'Native Art', 'Transavantgarde', 'Contemporary', 'Conceptual Art', 'Light and Space', 'Junk Art', 'Shin-hanga', 'Hard Edge Painting', 'Neo-Figurative Art', 'Purism', 'Tubism', 'Suprematism', 'Concretism', 'Analytical\xa0Realism', 'Mechanistic Cubism', 'Neo-Suprematism', 'Automatic Painting', 'Op Art', 'Minimalism', 'Post-Minimalism', 'Post-Painterly Abstraction', 'Neo-Concretism', 'Lettrism', 'Kinetic Art', 'New European Painting', 'P&D (Pattern and Decoration)', 'New Casualism', 'Neo-Dada', 'Spectralism', 'Rayonism', 'Synchromism', 'Modernism', 'Feminist Art', 'Transautomatism', 'Fantastic Realism', 'Photorealism', 'Hyper-Realism', 'Nouveau Réalisme', 'Postcolonial art', 'Sots Art', 'Indian Space painting', 'Zen', 'Spatialism', 'Cartographic Art', 'Superflat', 'Mail Art', 'Neo-Minimalism', 'Fiber art', 'Street art', 'Neo-Geo', 'Maximalism', 'Queer art', 'Digital Art', 'Cyber Art', 'Poster Art Realism', 'Hyper-Mannerism (Anachronism)', 'Confessional Art', 'Neo-Orthodoxism', 'Graffiti Art', 'Lowbrow Art', 'Stuckism']
    genres_to_drop = ['sketch and study', 'sculpture', 'design', 'installation', 'no genre', 'photo', 'poster', 'caricature', 'graffiti', 'advertisement', 'utensil', 'veduta', 'performance', 'capriccio', 'mural', 'bird-and-flower painting', 'digital', 'architecture', 'mobile', 'miniature', 'tapestry', 'pastorale', 'furniture', 'calligraphy', 'shan shui', 'mosaic', 'vanitas', 'jewelry', 'pin-up', 'video', "trompe-l'œil", 'panorama', 'stabile', 'augmented reality', 'quadratura', 'object', "artist's book", 'ornament', 'animation', 'tronie']

    #Clean the styles
    df_cleaned_styles = df[df['style'].isin(styles_to_keep)]

    #Clean the movement
    df_cleaned_styles_movements = df_cleaned_styles[df_cleaned_styles['movement'].isin(styles_to_keep)]

    #Create a copy of the dataframe and split the genres to a list (multiple values possibles)
    df_cleaned_styles_movements_copy = df_cleaned_styles_movements.copy()
    df_cleaned_styles_movements_copy['genre_list'] = df_cleaned_styles_movements_copy['genre'].str.split(',\s*', regex=True)
    df_cleaned_styles_movements_copy = df_cleaned_styles_movements_copy.drop(columns='genre')

    # Clean the genres (chosed the ones to drop)
    df_cleaned_styles_movement_genres = df_cleaned_styles_movements_copy[df_cleaned_styles_movements_copy['genre_list'].apply(
        lambda x: all(genre not in genres_to_drop for genre in x) if isinstance(x, list) else True
    )]

    df_filtered = df_cleaned_styles_movement_genres.drop_duplicates(subset=['img'], keep='first')
    df_filtered = df_filtered.set_index("file_name", drop=False)
    return df_filtered

In [7]:
df = filter_dataframe(df)

In [8]:
df.shape

(115055, 8)

In [9]:
df['style'].value_counts()

style
Impressionism         15301
Romanticism           13208
Realism               12730
Expressionism         10263
Post-Impressionism     7166
                      ...  
Geometric                 4
New Ink Painting          3
Kinetic Art               2
Rayonism                  1
Shin-hanga                1
Name: count, Length: 135, dtype: int64

In [10]:
counts = df['style'].value_counts()
rare_styles = counts[counts < 200]
print(rare_styles)

style
Neo-baroque          199
Classical Realism    198
Constructivism       190
Street art           188
Muralism             186
                    ... 
Geometric              4
New Ink Painting       3
Kinetic Art            2
Rayonism               1
Shin-hanga             1
Name: count, Length: 79, dtype: int64


In [11]:
df_filtered = df[df['style'].map(df['style'].value_counts()) > 200]

In [12]:
df_filtered.shape

(109618, 8)

In [13]:
df_filtered['style'].value_counts()

style
Impressionism                 15301
Romanticism                   13208
Realism                       12730
Expressionism                 10263
Post-Impressionism             7166
Surrealism                     6045
Symbolism                      4632
Abstract Expressionism         3457
Neoclassicism                  2306
Cubism                         2130
Pop Art                        2072
Art Nouveau (Modern)           1967
Art Informel                   1666
Magic Realism                  1552
Abstract Art                   1445
Contemporary Realism           1422
Neo-Expressionism              1339
Minimalism                     1223
Color Field Painting           1222
Orientalism                    1142
Lyrical Abstraction            1091
Fauvism                        1056
Naïve Art (Primitivism)         971
Neo-Impressionism               961
Academicism                     892
Op Art                          757
Naturalism                      707
Neo-Romanticism       

In [14]:
num_style = df_filtered['style'].nunique()
num_style

56

In [15]:
df_filtered['style'].unique()

array(['Neoclassicism', 'Orientalism', 'Romanticism', 'Academicism',
       'Kitsch', 'Realism', 'Naturalism', 'Impressionism',
       'Expressionism', 'Magic Realism', 'Symbolism', 'Biedermeier',
       'Luminism', 'Naïve Art (Primitivism)', 'Art Nouveau (Modern)',
       'Divisionism', 'Social Realism', 'Tonalism', 'Post-Impressionism',
       'Socialist Realism', 'Fauvism', 'Cubism', 'Abstract Art',
       'Precisionism', 'Figurative Expressionism', 'Surrealism',
       'Abstract Expressionism', 'Neo-Romanticism', 'Neo-Impressionism',
       'Regionalism', 'Art Deco', 'Neo-Expressionism',
       'Lyrical Abstraction', 'Contemporary Realism',
       'Color Field Painting', 'Pop Art', 'Dada', 'Synthetic Cubism',
       'Futurism', 'Tachisme', 'Metaphysical art', 'Art Informel',
       'Neo-Pop Art', 'Native Art', 'Transavantgarde', 'Contemporary',
       'Hard Edge Painting', 'Concretism', 'Op Art', 'Minimalism',
       'Conceptual Art', 'Post-Painterly Abstraction',
       'New Europ

In [16]:
df = df_filtered.copy()

In [17]:
df.shape

(109618, 8)

In [18]:
list_images= df['file_name'].unique().tolist()

# Trying to see the order of label encoder

In [19]:
list_style = ['Neoclassicism', 'Orientalism', 'Romanticism', 'Academicism',
       'Kitsch', 'Realism', 'Naturalism', 'Impressionism',
       'Expressionism', 'Magic Realism', 'Symbolism', 'Biedermeier',
       'Luminism', 'Naïve Art (Primitivism)', 'Art Nouveau (Modern)',
       'Divisionism', 'Social Realism', 'Tonalism', 'Post-Impressionism',
       'Socialist Realism', 'Fauvism', 'Cubism', 'Abstract Art',
       'Precisionism', 'Figurative Expressionism', 'Surrealism',
       'Abstract Expressionism', 'Neo-Romanticism', 'Neo-Impressionism',
       'Regionalism', 'Art Deco', 'Neo-Expressionism',
       'Lyrical Abstraction', 'Contemporary Realism',
       'Color Field Painting', 'Pop Art', 'Dada', 'Synthetic Cubism',
       'Futurism', 'Tachisme', 'Metaphysical art', 'Art Informel',
       'Neo-Pop Art', 'Native Art', 'Transavantgarde', 'Contemporary',
       'Hard Edge Painting', 'Concretism', 'Op Art', 'Minimalism',
       'Conceptual Art', 'Post-Painterly Abstraction',
       'New European Painting', 'Neo-Dada', 'Fantastic Realism',
       'Hyper-Realism']

In [24]:
list_paths = []
lists_of_style = []

for style in list_style:
    row = df.loc[df['style'] == style].iloc[0]
    list_paths.append(row.file_name)
    lists_of_style.append(row.style)

In [25]:
list_paths

['28564-the-waterfalls-at-tivoli.jpg',
 '30032-thomas-keymer-of-kidwelly-la-chinoise-1754.jpg',
 '30186-self-portrait-at-the-age-of-about-twenty.jpg',
 '32054-ptoleme2-vincenzo-camuccini-1813.jpg',
 '33447-the-song-of-the-lark-1903.jpg',
 '33572-daughter-of-fisherman-1872.jpg',
 '33579-the-spinner-1873.jpg',
 '34315-p-084-d-1-k-9-1.jpg',
 '34338-man-with-a-monkey-1915.jpg',
 '34396-carnevale-1923.jpg',
 '35496-oberon-titania-and-puck-with-fairies-dancing(1).jpg',
 '38887-the-day-laborer-with-his-son-1823.jpg',
 '39263-a-pastoral-scene.jpg',
 '41742-dr-john-sappington-1834.jpg',
 '47890-lotz-k-roly-korn-lia.jpg',
 '48671-young-woman-at-the-piano.jpg',
 '50380-roll-call-1874.jpg',
 '50547-blakelock-ralph-albert-fifty-ninth-st-and-fifth-ave-ca-1868-70.jpg',
 '52462-morning-at-the-beach-by-fujishima-takeji-geidai-museum.jpg',
 '75860-foreigners-in-leningrad-1937.jpg',
 '76831-enchantment-of-the-sun-1905.jpg',
 '77498-cubist-composition-1920(1).jpg',
 '77503-cubist-composition-1920-1.jpg',


In [26]:
len(list_paths)

56

In [29]:
len(lists_of_style)

56

In [30]:
len(set(lists_of_style))

56

In [33]:
df_test = pd.DataFrame({"lists_of_style" : lists_of_style, "list_paths" : list_paths})

In [34]:
df_test

,lists_of_style,list_paths
0,Neoclassicism,28564-the-waterfalls-at-tivoli.jpg
1,Orientalism,30032-thomas-keymer-of-kidwelly-la-chinoise-17...
2,Romanticism,30186-self-portrait-at-the-age-of-about-twenty...
3,Academicism,32054-ptoleme2-vincenzo-camuccini-1813.jpg
4,Kitsch,33447-the-song-of-the-lark-1903.jpg
5,Realism,33572-daughter-of-fisherman-1872.jpg
6,Naturalism,33579-the-spinner-1873.jpg
7,Impressionism,34315-p-084-d-1-k-9-1.jpg
8,Expressionism,34338-man-with-a-monkey-1915.jpg
9,Magic Realism,34396-carnevale-1923.jpg


In [35]:
df_test["filepath"] = df_test["list_paths"].apply(
    lambda x: os.path.join("/kaggle/input/wikiart-all-artpieces/wikiart/wikiart", x)
)
df_test

,lists_of_style,list_paths,filepath
0,Neoclassicism,28564-the-waterfalls-at-tivoli.jpg,/kaggle/input/wikiart-all-artpieces/wikiart/wi...
1,Orientalism,30032-thomas-keymer-of-kidwelly-la-chinoise-17...,/kaggle/input/wikiart-all-artpieces/wikiart/wi...
2,Romanticism,30186-self-portrait-at-the-age-of-about-twenty...,/kaggle/input/wikiart-all-artpieces/wikiart/wi...
3,Academicism,32054-ptoleme2-vincenzo-camuccini-1813.jpg,/kaggle/input/wikiart-all-artpieces/wikiart/wi...
4,Kitsch,33447-the-song-of-the-lark-1903.jpg,/kaggle/input/wikiart-all-artpieces/wikiart/wi...
5,Realism,33572-daughter-of-fisherman-1872.jpg,/kaggle/input/wikiart-all-artpieces/wikiart/wi...
6,Naturalism,33579-the-spinner-1873.jpg,/kaggle/input/wikiart-all-artpieces/wikiart/wi...
7,Impressionism,34315-p-084-d-1-k-9-1.jpg,/kaggle/input/wikiart-all-artpieces/wikiart/wi...
8,Expressionism,34338-man-with-a-monkey-1915.jpg,/kaggle/input/wikiart-all-artpieces/wikiart/wi...
9,Magic Realism,34396-carnevale-1923.jpg,/kaggle/input/wikiart-all-artpieces/wikiart/wi...


In [37]:
le = LabelEncoder()
y_encoded = le.fit_transform(df_test['lists_of_style'])

In [38]:
y_encoded

array([37, 40, 47,  2, 23, 45, 30, 22, 15, 26, 51,  6, 24, 31,  5, 14, 48,
       54, 42, 49, 17, 12,  0, 44, 18, 50,  1, 36, 34, 46,  3, 33, 25, 11,
        7, 41, 13, 52, 19, 53, 27,  4, 35, 29, 55, 10, 20,  9, 39, 28,  8,
       43, 38, 32, 16, 21])

In [39]:
from tensorflow import keras

# Charger le modèle
model = keras.models.load_model("/kaggle/input/model-before-finetuning/model_Xception_alldata.keras")

I0000 00:00:1757408716.636914      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1757408716.637707      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [40]:
images_paths = df_test['filepath'].tolist()

In [41]:
images_paths

['/kaggle/input/wikiart-all-artpieces/wikiart/wikiart/28564-the-waterfalls-at-tivoli.jpg',
 '/kaggle/input/wikiart-all-artpieces/wikiart/wikiart/30032-thomas-keymer-of-kidwelly-la-chinoise-1754.jpg',
 '/kaggle/input/wikiart-all-artpieces/wikiart/wikiart/30186-self-portrait-at-the-age-of-about-twenty.jpg',
 '/kaggle/input/wikiart-all-artpieces/wikiart/wikiart/32054-ptoleme2-vincenzo-camuccini-1813.jpg',
 '/kaggle/input/wikiart-all-artpieces/wikiart/wikiart/33447-the-song-of-the-lark-1903.jpg',
 '/kaggle/input/wikiart-all-artpieces/wikiart/wikiart/33572-daughter-of-fisherman-1872.jpg',
 '/kaggle/input/wikiart-all-artpieces/wikiart/wikiart/33579-the-spinner-1873.jpg',
 '/kaggle/input/wikiart-all-artpieces/wikiart/wikiart/34315-p-084-d-1-k-9-1.jpg',
 '/kaggle/input/wikiart-all-artpieces/wikiart/wikiart/34338-man-with-a-monkey-1915.jpg',
 '/kaggle/input/wikiart-all-artpieces/wikiart/wikiart/34396-carnevale-1923.jpg',
 '/kaggle/input/wikiart-all-artpieces/wikiart/wikiart/35496-oberon-titania

In [42]:
images_list = []

for img_path in images_paths :
            img = Image.open(img_path).convert('RGB')

            img_resized = img.resize((224, 224), Image.BICUBIC)

            img_array = np.array(img_resized)

            image_preprocessed = preprocess_input(img_array)
            # convert into an array and add to list
            images_list.append(image_preprocessed)

images_np = np.stack(images_list)

In [43]:
preds = model.predict(images_np)
print(preds)

I0000 00:00:1757408909.446731     128 service.cc:148] XLA service 0x7d7fb0009d60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1757408909.447459     128 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1757408909.447480     128 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1757408909.897847     128 cuda_dnn.cc:529] Loaded cuDNN version 90300


1/2 ━━━━━━━━━━━━━━━━━━━━ 10s 11s/step

I0000 00:00:1757408918.020820     128 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2/2 ━━━━━━━━━━━━━━━━━━━━ 17s 7s/step
[[3.3544522e-05 1.1121537e-04 1.7831618e-02 ... 9.5509295e-06
  1.1949792e-03 4.4926396e-04]
 [1.7798984e-06 6.2302793e-06 5.0462417e-02 ... 7.5670710e-07
  2.3369455e-04 1.1706069e-04]
 [1.6117700e-06 3.4023235e-06 6.3638635e-02 ... 2.2530649e-07
  2.0899709e-04 1.2427708e-04]
 ...
 [3.0467852e-03 7.8520561e-03 4.8261425e-03 ... 6.7353452e-04
  2.1907741e-04 2.6175447e-02]
 [1.8759086e-03 7.8792358e-03 4.0078234e-05 ... 2.1255733e-03
  2.0601517e-04 1.5466553e-03]
 [1.5488118e-03 5.3687464e-03 2.5406266e-03 ... 1.0046394e-03
  9.4434439e-04 6.9874944e-03]]


In [44]:
# indices
pred_indices = preds.argmax(axis=1)

In [60]:
list_style[pred_indices[0]]

'Romanticism'

In [46]:
pred_labels = le.inverse_transform(pred_indices)
pred_labels

array(['Romanticism', 'Romanticism', 'Neoclassicism', 'Romanticism',
       'Academicism', 'Academicism', 'Academicism', 'Impressionism',
       'Neoclassicism', 'Realism', 'Symbolism', 'Romanticism',
       'Romanticism', 'Realism', 'Impressionism', 'Expressionism',
       'Realism', 'Impressionism', 'Impressionism', 'Expressionism',
       'Expressionism', 'Cubism', 'Abstract Art', 'Precisionism',
       'Expressionism', 'Surrealism', 'Expressionism', 'Impressionism',
       'Expressionism', 'Expressionism', 'Expressionism', 'Surrealism',
       'Realism', 'Post-Impressionism', 'Expressionism', 'Cubism',
       'Surrealism', 'Cubism', 'Cubism', 'Abstract Expressionism',
       'Post-Impressionism', 'Surrealism', 'Surrealism', 'Expressionism',
       'Expressionism', 'Surrealism', 'Hard Edge Painting', 'Surrealism',
       'Hard Edge Painting', 'Minimalism', 'Pop Art',
       'Abstract Expressionism', 'Expressionism', 'Surrealism',
       'Post-Impressionism', 'Contemporary Realism'],

In [49]:
df_test['predicted_class'] = pred_labels

In [50]:
df_test

,lists_of_style,list_paths,filepath,predicted_class
0,Neoclassicism,28564-the-waterfalls-at-tivoli.jpg,/kaggle/input/wikiart-all-artpieces/wikiart/wi...,Romanticism
1,Orientalism,30032-thomas-keymer-of-kidwelly-la-chinoise-17...,/kaggle/input/wikiart-all-artpieces/wikiart/wi...,Romanticism
2,Romanticism,30186-self-portrait-at-the-age-of-about-twenty...,/kaggle/input/wikiart-all-artpieces/wikiart/wi...,Neoclassicism
3,Academicism,32054-ptoleme2-vincenzo-camuccini-1813.jpg,/kaggle/input/wikiart-all-artpieces/wikiart/wi...,Romanticism
4,Kitsch,33447-the-song-of-the-lark-1903.jpg,/kaggle/input/wikiart-all-artpieces/wikiart/wi...,Academicism
5,Realism,33572-daughter-of-fisherman-1872.jpg,/kaggle/input/wikiart-all-artpieces/wikiart/wi...,Academicism
6,Naturalism,33579-the-spinner-1873.jpg,/kaggle/input/wikiart-all-artpieces/wikiart/wi...,Academicism
7,Impressionism,34315-p-084-d-1-k-9-1.jpg,/kaggle/input/wikiart-all-artpieces/wikiart/wi...,Impressionism
8,Expressionism,34338-man-with-a-monkey-1915.jpg,/kaggle/input/wikiart-all-artpieces/wikiart/wi...,Neoclassicism
9,Magic Realism,34396-carnevale-1923.jpg,/kaggle/input/wikiart-all-artpieces/wikiart/wi...,Realism


In [51]:
list_style = sorted(['Neoclassicism', 'Orientalism', 'Romanticism', 'Academicism',
       'Kitsch', 'Realism', 'Naturalism', 'Impressionism',
       'Expressionism', 'Magic Realism', 'Symbolism', 'Biedermeier',
       'Luminism', 'Naïve Art (Primitivism)', 'Art Nouveau (Modern)',
       'Divisionism', 'Social Realism', 'Tonalism', 'Post-Impressionism',
       'Socialist Realism', 'Fauvism', 'Cubism', 'Abstract Art',
       'Precisionism', 'Figurative Expressionism', 'Surrealism',
       'Abstract Expressionism', 'Neo-Romanticism', 'Neo-Impressionism',
       'Regionalism', 'Art Deco', 'Neo-Expressionism',
       'Lyrical Abstraction', 'Contemporary Realism',
       'Color Field Painting', 'Pop Art', 'Dada', 'Synthetic Cubism',
       'Futurism', 'Tachisme', 'Metaphysical art', 'Art Informel',
       'Neo-Pop Art', 'Native Art', 'Transavantgarde', 'Contemporary',
       'Hard Edge Painting', 'Concretism', 'Op Art', 'Minimalism',
       'Conceptual Art', 'Post-Painterly Abstraction',
       'New European Painting', 'Neo-Dada', 'Fantastic Realism',
       'Hyper-Realism'])
list_style

['Abstract Art',
 'Abstract Expressionism',
 'Academicism',
 'Art Deco',
 'Art Informel',
 'Art Nouveau (Modern)',
 'Biedermeier',
 'Color Field Painting',
 'Conceptual Art',
 'Concretism',
 'Contemporary',
 'Contemporary Realism',
 'Cubism',
 'Dada',
 'Divisionism',
 'Expressionism',
 'Fantastic Realism',
 'Fauvism',
 'Figurative Expressionism',
 'Futurism',
 'Hard Edge Painting',
 'Hyper-Realism',
 'Impressionism',
 'Kitsch',
 'Luminism',
 'Lyrical Abstraction',
 'Magic Realism',
 'Metaphysical art',
 'Minimalism',
 'Native Art',
 'Naturalism',
 'Naïve Art (Primitivism)',
 'Neo-Dada',
 'Neo-Expressionism',
 'Neo-Impressionism',
 'Neo-Pop Art',
 'Neo-Romanticism',
 'Neoclassicism',
 'New European Painting',
 'Op Art',
 'Orientalism',
 'Pop Art',
 'Post-Impressionism',
 'Post-Painterly Abstraction',
 'Precisionism',
 'Realism',
 'Regionalism',
 'Romanticism',
 'Social Realism',
 'Socialist Realism',
 'Surrealism',
 'Symbolism',
 'Synthetic Cubism',
 'Tachisme',
 'Tonalism',
 'Transavan

In [61]:
le.inverse_transform(np.array([47]))

array(['Romanticism'], dtype=object)

#### Loading and resize images ####

In [ ]:
#create full path column in df
df["filepath"] = df["file_name"].apply(
    lambda x: os.path.join("/kaggle/input/wikiart-all-artpieces/wikiart/wikiart", x)
)
df.head()

In [ ]:
df_test = df.head(10)

#### Define X and y ####

In [ ]:
X = df_test["filepath"]
y = df_test['style']

In [ ]:
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

#### Encode labels ####

In [ ]:
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

# Create tensorflow dataset

In [ ]:
NUM_CLASSES = len(le.classes_)
NUM_CLASSES

In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
def is_valid_image(path):
    """Retourne True si l'image peut être décodée, False sinon"""
    try:
        img = read_file(path)
        decode_jpeg(img, channels=3)
        return True
    except:
        return False

def process_image(path, label):
    """Lit, redimensionne, normalise l'image et encode le label en one-hot"""
    img = read_file(path)
    img = decode_jpeg(img, channels=3)
    img = resize(img, IMG_SIZE)
    img = preprocess_input(img)
    
    label_one_hot = one_hot(label, depth=NUM_CLASSES)
    label_one_hot = cast(label_one_hot, float32)
    
    return img, label_one_hot

def filter_valid_images(paths, labels):
    """Filtre les images corrompues et retourne les listes valides"""
    valid_paths, valid_labels = [], []
    invalid_paths = []
    
    for p, l in zip(paths, labels):
        if is_valid_image(p):
            valid_paths.append(p)
            valid_labels.append(l)
        else:
            invalid_paths.append(p)
    
    print(f"⚠️ Images invalides : {len(invalid_paths)}")
    if invalid_paths:
        print(invalid_paths[:10])  # montrer les premières images corrompues
    
    return valid_paths, valid_labels

In [ ]:
# filter valid images
X_train_valid, y_train_valid = filter_valid_images(X_train, y_train_encoded)
X_test_valid, y_test_valid = filter_valid_images(X_test, y_test_encoded)

In [ ]:
# Create tensorflow dataset secured
train_dataset = Dataset.from_tensor_slices((X_train_valid, y_train_valid))
train_dataset = train_dataset.map(process_image, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.shuffle(1000).batch(BATCH_SIZE).prefetch(AUTOTUNE)

val_dataset = Dataset.from_tensor_slices((X_val_valid, y_val_valid))
val_dataset = val_dataset.map(process_image, num_parallel_calls=AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)

In [ ]:
train_dataset.head()

# Load model

In [1]:
from tensorflow import keras

# Charger le modèle
model = keras.models.load_model("/kaggle/input/model-before-finetuning/model_Xception_alldata.keras")

2025-09-09 07:52:45.229839: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757404365.516547      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757404365.603357      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-09 07:53:04.798582: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


### finetuning

In [2]:
for i, layer in enumerate(model.layers):
    print(i, layer.name, layer.trainable)

0 input_layer_4 True
1 block1_conv1 False
2 block1_conv1_bn False
3 block1_conv1_act False
4 block1_conv2 False
5 block1_conv2_bn False
6 block1_conv2_act False
7 block2_sepconv1 False
8 block2_sepconv1_bn False
9 block2_sepconv2_act False
10 block2_sepconv2 False
11 block2_sepconv2_bn False
12 conv2d_16 False
13 block2_pool False
14 batch_normalization_16 False
15 add_48 False
16 block3_sepconv1_act False
17 block3_sepconv1 False
18 block3_sepconv1_bn False
19 block3_sepconv2_act False
20 block3_sepconv2 False
21 block3_sepconv2_bn False
22 conv2d_17 False
23 block3_pool False
24 batch_normalization_17 False
25 add_49 False
26 block4_sepconv1_act False
27 block4_sepconv1 False
28 block4_sepconv1_bn False
29 block4_sepconv2_act False
30 block4_sepconv2 False
31 block4_sepconv2_bn False
32 conv2d_18 False
33 block4_pool False
34 batch_normalization_18 False
35 add_50 False
36 block5_sepconv1_act False
37 block5_sepconv1 False
38 block5_sepconv1_bn False
39 block5_sepconv2_act False
40 b

In [3]:
for layer in model.layers[100:132]:  # Excluding the layers we add ourself
    layer.trainable = True


In [4]:
for i, layer in enumerate(model.layers):
    print(i, layer.name, layer.trainable)

0 input_layer_4 True
1 block1_conv1 False
2 block1_conv1_bn False
3 block1_conv1_act False
4 block1_conv2 False
5 block1_conv2_bn False
6 block1_conv2_act False
7 block2_sepconv1 False
8 block2_sepconv1_bn False
9 block2_sepconv2_act False
10 block2_sepconv2 False
11 block2_sepconv2_bn False
12 conv2d_16 False
13 block2_pool False
14 batch_normalization_16 False
15 add_48 False
16 block3_sepconv1_act False
17 block3_sepconv1 False
18 block3_sepconv1_bn False
19 block3_sepconv2_act False
20 block3_sepconv2 False
21 block3_sepconv2_bn False
22 conv2d_17 False
23 block3_pool False
24 batch_normalization_17 False
25 add_49 False
26 block4_sepconv1_act False
27 block4_sepconv1 False
28 block4_sepconv1_bn False
29 block4_sepconv2_act False
30 block4_sepconv2 False
31 block4_sepconv2_bn False
32 conv2d_18 False
33 block4_pool False
34 batch_normalization_18 False
35 add_50 False
36 block5_sepconv1_act False
37 block5_sepconv1 False
38 block5_sepconv1_bn False
39 block5_sepconv2_act False
40 b

In [ ]:
# Recompiler avec un learning rate plus petit
model.compile(optimizer=optimizers.Adam(1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

es = EarlyStopping(monitor = 'val_loss',
                   patience = 10,
                   verbose = 1,
                   restore_best_weights = True)

history_phase2 = model.fit(train_dataset,
                           validation_data=val_dataset,
                           epochs=100,
                           callbacks=[es])

In [ ]:
model.save("/kaggle/working/model_Xception_alldata_finetuned.keras")

In [ ]:
model.evaluate(X_test, y_test_cat, return_dict=True)

In [ ]:
plot_history(history_phase2)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test_cat, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

In [ ]:
images_paths = ["/kaggle/input/images-to-try/Paul_Czanne_-_The_Basket_of_Apples_-_1926.252_-_Art_Institute_of_Chicago.jpg",
                "/kaggle/input/images-to-try/Piet_Mondriaan_1942_-_New_York_City_I.jpg",
                "/kaggle/input/images-to-try/Two_Young_Girls_at_the_Piano_MET_rl1975.1.201.R.jpg",
                "/kaggle/input/images-to-try/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg"]

In [ ]:
images_list = []

for img_path in images_paths :
            img = Image.open(img_path).convert('RGB')

            img_resized = img.resize((224, 224), Image.BICUBIC)

            img_array = np.array(img_resized)

            image_preprocessed = preprocess_input(img_array)
            # convert into an array and add to list
            images_list.append(image_preprocessed)

images_np = np.stack(images_list)

In [ ]:
X_eval = images_np

In [ ]:
preds = model.predict(X_eval)
print(preds)

In [ ]:
# indices
pred_indices = preds.argmax(axis=1)
# label
pred_labels = le.inverse_transform(pred_indices)
pred_labels

In [ ]:
for img_name, label, pred in zip(images_paths, pred_labels, preds):
    print(f"{img_name} : {label} ({pred.max():.2f})")